#### **Mini-Project : Currency Conversion Tool**

Creating a Custom tool to convert x amount of base currency to y amount of Target Currency 

In [34]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
import requests

from langchain_core.tools import InjectedToolArg
from typing import Annotated
import json

load_dotenv()

python-dotenv could not parse statement starting at line 3


True

**InjectedToolArg-** Tells the LLM- do not try to fill this argument, I will inject this value after running earlier tools.

In [28]:
# Step 1 :Tool Creation
@tool 
def get_currency_factor(base_currency: str, target_currency: str) -> float:
    """
  This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

    response = requests.get(url)

    return response.json()


@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    This function converts a given amount of base currency to the target currency using a provided conversion rate.
    """
    return base_currency_value * conversion_rate

In [29]:
get_currency_factor.invoke({"base_currency": "USD", "target_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1763769602,
 'time_last_update_utc': 'Sat, 22 Nov 2025 00:00:02 +0000',
 'time_next_update_unix': 1763856002,
 'time_next_update_utc': 'Sun, 23 Nov 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 89.4451}

In [30]:
convert.invoke({"base_currency_value": 25, "conversion_rate":89.4451})

2236.1275

In [31]:
# Step 2 : Tool Binding

model = ChatGroq(model="llama-3.3-70b-versatile")

model_tools = model.bind_tools([get_currency_factor, convert])

In [33]:
messages = [HumanMessage("What is the conversion factor between INR and USD and based on that can you convert 10 USD to inr")]

ai_message = model_tools.invoke(messages)
print(ai_message.tool_calls)

messages.append(ai_message)

[{'name': 'get_currency_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': '0bv0d8kkz', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': 'f5vc1kzv6', 'type': 'tool_call'}]


In [38]:
# Step 3 : Tool Execution

for tool_call in ai_message.tool_calls:
    # Executing Tool 1 and getting the value of conversion rate
    if tool_call["name"] == "get_currency_factor":
        tool_message1 = get_currency_factor.invoke(tool_call)
        # Fetch the value of conversion rate from the tool message
        conversion_rate = json.loads(tool_message1.content)["conversion_rate"]  # converting the string to json (dictionary) because the tool message is a string
        
        # Append this tool message to messages list
        messages.append(tool_message1)

    if tool_call["name"] == "convert":
        tool_call["args"]["conversion_rate"] = conversion_rate  # Appending the conversion rate to the tool call arguments
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [ ]:
# Step 4 : LLM Response including the answer from tools

model_tools.invoke(messages).content

'The conversion factor between INR and USD is 89.4451. Based on this conversion factor, 10 USD is equivalent to 894.451 INR.'

: 